In [22]:
import csv
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tqdm._tqdm_notebook import tqdm_notebook
import sklearn.model_selection
STOPWORDS = set(stopwords.words('english'))

In [23]:
vocab_size = 45000
embedding_dim = 512
max_length = 100
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
training_portion = .8

In [24]:
dataset = pd.read_csv('classifier_data_0.csv', header = 0, encoding= 'utf-8')
dataset.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9', 'Unnamed: 10'], axis=1, inplace = True)

In [25]:
X= dataset['description']
y = dataset['owner']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1500)
X_train.shape
df = pd.DataFrame(data=[X_train, y_train], index=["description", "owner"]).T
df = df.append(pd.DataFrame(data=[X_test, y_test], index=["description", "owner"]).T)

print(df.head())

                                            description  \
4576  product version       see aboutversionurls if ...   
3457  Version: 32.0.1653.0 (Official Build 225591) c...   
3879  When I try to add my self to review a code in ...   
788   Currently enter_chroot.sh has a hack to preser...   
307   Product Version      : <see about:version>URLs...   

                        owner  
4576                      NaN  
3457      kinaba@chromium.org  
3879    nsylvain@chromium.org  
788   davidjames@chromium.org  
307          jon@chromium.org  


In [26]:

df['char_count'] = df['description'].apply(len) # Number of characters in the string
df['word_count'] = df['description'].apply(lambda x: len(x.split())) # Number of words in the string 
df['word_density'] = df['char_count'] / (df['word_count']+1) # Density of word (in char)
df['title_word_count'] = df['description'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))

In [27]:
train_x, valid_x, y_train, y_valid = model_selection.train_test_split(df['description'], df['owner'], random_state=42, stratify=df['owner'], test_size=0.2)

token = Tokenizer()
token.fit_on_texts(df['description'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=300)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=300)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = pretrained.get_word_vector(word) #embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        

def create_conv_model(word_index, label, embedding_matrix, pre_trained=False):
    '''
    Function to generate a convulational neural network for binary or multiclass classification.
    @param word_index: (matrix) unique token in corpus
    @param label: (list) list of labels to determine if it,s a binary or multiclass
    @param embedding_matrix: (matrix) matrix of integer for each word in the 
    @param pre_trained: (bool) determine if the model will use pretrained model
    @return: (model) convulational neural network 
    '''
    if pre_trained==False:
        embedded = keras.layers.Embedding(len(word_index) +1, 100)
    else:
        embedded = keras.layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)
    
    model = keras.Sequential([
    embedded,
    keras.layers.Conv1D(100, 5, activation='relu'), # padding='same'
    keras.layers.Dropout(0.2),
    keras.layers.MaxPooling1D(pool_size=4),
    keras.layers.Conv1D(64, 5, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.MaxPooling1D(pool_size=4),
    keras.layers.Conv1D(32, 5, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.GlobalMaxPooling1D(),

    keras.layers.Dense(1 if len(label)<=2 else len(label), activation='sigmoid' if len(label)<=2 else "softmax")])

    if len(label)==2:
        model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=1e-4),
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
    else: 
        model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=1e-4),
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    return model

NameError: name 'model_selection' is not defined

In [ ]:
model = create_conv_model(word_index, labels, pre_trained=pre_trained)

In [ ]:
 history = model.fit(train_seq_x, train_y,
                    epochs=50, callbacks=[es],
                    validation_split=0.2)

In [ ]:
results = model.evaluate(valid_seq_x, valid_y)

    print(results)
    print(f"\nThe precision of the model is {round(100*precision_score(valid_y, (model.predict(valid_seq_x)>0.5).astype(int), labels),2)}%")